In [1]:
import degirum as dg, degirum_tools
#import os
from pathlib import Path
import gen_test_image_util as imgutil
#import importlib
import os
import time
import cv2
import yaml
#import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
#importlib.reload(imgutil)

In [2]:
with open('./benchmark_200pics_mobilenet_two_orca.yaml') as file:
    obj = yaml.safe_load(file)
    hw_location = obj['hw_location']
    orca1_zoo_url = obj['orca1_zoo_url']
    orca_zoo_url = obj['orca_zoo_url']
    exclude_preprocessing = obj['exclude_preprocessing']
    use_jpeg = obj['use_jpeg']
    image_dir = Path(obj['image_dir'])
    num_of_pics = obj['num_of_pics']
    num_of_orca = obj['num_of_orca']
    batch_size = obj['batch_size']
    image_size = obj['image_size']

In [3]:
def partial_transaction(model_instance, image_chunk_pathL, start_index, num_of_pics):
    
    count = 0
   
    try:
        for res in model_instance.predict_batch(imgutil.pics_transaction_gen(model_instance, image_chunk_pathL, exclude_preprocessing, start_index, num_of_pics)):
            count += 1
            #pass
    except Exception as e:
        print(e)
    
    #print(f'count:{count}')
    return count

In [4]:
imgutil.remove_image_chunk(image_dir, 3000)
image_chunkL = imgutil.create_test_image_chunk('maami_1.jpg', image_size)
image_chunk_pathL = imgutil.save_image_chunk(image_dir, image_chunkL)
    
orca_zoo = dg.connect(hw_location, orca_zoo_url, degirum_tools.get_token()) # connect to AI inference engine
orca1_zoo = dg.connect(hw_location, orca1_zoo_url, degirum_tools.get_token()) # connect to AI inference engine

model_orca = 'mobilenet_v1_imagenet--224x224_quant_n2x_orca_1'
model_orca1 = 'mobilenet_v1_imagenet--224x224_quant_n2x_orca1_1'   
#model_orca = 'mobilenet_v2_imagenet--224x224_quant_n2x_orca_1'
#model_orca1 = 'mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1'   


durationL = []
          
image = cv2.imread(image_chunk_pathL[0], cv2.IMREAD_COLOR)
h,w = image.shape[:2]
image_size[0] = h
image_size[1] = w

instance_orca = imgutil.create_model_instances(model_orca, orca_zoo, batch_size, use_jpeg)
instance_orca1 = imgutil.create_model_instances(model_orca1, orca1_zoo, batch_size, use_jpeg)

countL = []
resultL = []
        
#single device orca
instance_orca(image_chunk_pathL[0]) #dummy
instance_orca.reset_time_stats()

t = degirum_tools.Timer()
count = partial_transaction(instance_orca, image_chunk_pathL, 0, num_of_pics)
orca_elapsed_time = round(1e3*t(),1)
if not count == num_of_pics:
    print("count: {count}, error: ORCA single model")
    exit()    

#single device orca1
instance_orca1(image_chunk_pathL[int(num_of_pics/num_of_orca)]) #dummy
instance_orca.reset_time_stats()

t = degirum_tools.Timer()
count = partial_transaction(instance_orca1, image_chunk_pathL, 0, num_of_pics)
orca1_elapsed_time = round(1e3*t(),1)
if not count == num_of_pics:
    print("count: {count}, error: ORCA1 single model")
    exit()    

#two devices orca + orca1
instance_orca(image_chunk_pathL[0]) #dummy
instance_orca1(image_chunk_pathL[int(num_of_pics/num_of_orca)]) #dummy

instance_orca.reset_time_stats()
instance_orca1.reset_time_stats()

t = degirum_tools.Timer()
with ThreadPoolExecutor(max_workers = 2) as exctor:
    try:
        count_orca = exctor.submit(partial_transaction, instance_orca, image_chunk_pathL, 0, int(num_of_pics/num_of_orca))
        count_orca1 = exctor.submit(partial_transaction, instance_orca1, image_chunk_pathL, int(num_of_pics/num_of_orca), int(num_of_pics/num_of_orca)) 
    except Exception as e:
        print(f'Error:{e}') 
    
    resultL = [count_orca.result(), count_orca1.result()]   
    #print(f'resultL = {resultL}')              
    two_orca_elapsed_time = round(1e3*t(),1)  
    #print(f'count:{sum(resultL)}')
    
    if not(sum(resultL) == num_of_pics): 
        print(f"count: {sum(resultL)}, error: ORCA + ORCA1")
        exit()    


DegirumException: zoo 'orca_models' not found. (cloud server response: 400 Client Error: Bad Request for url: https://cs.degirum.com/zoo/v1/public/models/degirum/orca_models)

In [ ]:
#print results    
CW = (50, 20, 30)  # column widths
header = f"| {'Model name':{CW[0]}}| {'Device':{CW[1]}}| {'200 Images Inference Time[ms]':{CW[2]}}|"

print('200 images transaction demo (Single device, Two devices)\n')
print(f"hw_location:  {hw_location}")
print(f"Batch Size:   {batch_size}")
print(f"Image Size:   {image_size[0]} x {image_size[1]}")
print(f"{'-'*len(header)}")
print(header)
print(f"{'-'*len(header)}")

print(f"| {model_orca:{CW[0]}}|" + f"{' ORCA':{CW[1]}} |" + f" {orca_elapsed_time:{CW[2]}}|")
print(f"| {model_orca1:{CW[0]}}|" + f"{' ORCA1':{CW[1]}} |" + f" {orca1_elapsed_time:{CW[2]}}|")
print(f"| {model_orca1:{CW[0]}}|" + f"{' ORCA + ORCA1':{CW[1]}} |" + f" {two_orca_elapsed_time:{CW[2]}}|")  
    
print(f"{'-'*len(header)}") 